# IterativeEikonal
Approximate the viscosity solution of the  Eikonal equation 
$$\begin{cases} \Vert \nabla_{\mathrm{cost}} W(g) \Vert_{\mathrm{cost}} = 1, & g \in G \setminus S, \\
W(g) = 0, & g \in S \subset G. \end{cases}$$
This method is based on the paper ["A PDE Approach to Data-Driven Sub-Riemannian Geodesics in $SE(2)$" (2015) by E. J. Bekkers, R. Duits, A. Mashtakov, and G. R. Sanguinetti](https://doi.org/10.1137/15M1018460).

In [ ]:
import eikivp
import taichi as ti
ti.init(arch=ti.gpu, debug=False)
import numpy as np
import scipy as sp
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

## $\mathbb{R}^2$

### Flat space
In fact, we will solve the Eikonal equation on some subset $G \subset \mathbb{R}^2$, namely $G = [-1, 1] \times [-2, 2]$. The source set will consist of a single point: $S = \{(0, 0)\}$. Then we know that the viscosity solution is simply the Euclidean norm. 

There are numerous ways to numerically solve the Eikonal equation. We could use [Fast Marching](https://en.wikipedia.org/wiki/Fast_marching_method). This is an efficient method, but it requires a lot of work to make it compatible with non-Euclidean domains. 

#### Distance map
Here we define the coordinate system.

In [ ]:
x_min, x_max = -2, 2
y_min, y_max = -1, 1
N = 501
dim_J = 2*N-1
dim_I = N
if (x_max - x_min) / (dim_J - 1) == (y_max - y_min) / (dim_I - 1):
    dxy = (x_max - x_min) / (dim_J - 1)
else:
    raise ValueError("Spatial resolution in x- and y-directions is not the same.")
Is, Js = np.indices((dim_I, dim_J))
xs, ys = eikivp.R2.metric.coordinate_array_to_real(Is, Js, x_min, y_max, dxy)

In [ ]:
source_point_real = (0., 0.)
source_point = eikivp.R2.metric.coordinate_real_to_array(*source_point_real, x_min, y_max, dxy)
source_point_array = (np.logical_and(xs == source_point_real[0], ys == source_point_real[1])) # 1 at source point, 0 else.
G_np = np.diag((1., 4.)) # np.array(((2., -1.), (-1., 2.))) np.identity(2)
cost = np.ones((dim_I, dim_J))

The exact solution of the Eikonal PDE is easy to compute in this case: it is given by
$$W(x, y) = (g_{xx} x^2 + \underbrace{g_{xy} x y + g_{yx} y x}_{= 2 g_{xy} x y} + g_{yy} y^2)^{\frac{1}{2}}.$$

In [ ]:
W_exact = np.sqrt(G_np[0, 0] * xs ** 2 + 2 * G_np[0, 1] * xs * ys + G_np[1, 1] * ys ** 2)

Next, we apply the IVP approach.

In [ ]:
n = 5000
W_R2_flat, grad_W_R2_flat = eikivp.eikonal_solver_R2(cost, source_point, G_np=G_np, dxy=dxy, n_max=n)

Finally, we can compute the distance map using Fast Marching. This is a very naïve method written in pure Python (loops in loops in loops🙃), so it becomes very slow if the domain larger than say 200 x 400. Moreover, it only works when the metric tensor is given by the identity matrix.

In [ ]:
do_fast_marching = N < 250 and np.all(G_np == np.identity(2))
do_fast_marching

In [ ]:
def fast_marching_R2(f, source_point, dxy):
    """
    Naive implementation of fast marching to solve Eikonal equation on R2 with 
    cost `f`."""
    shape = f.shape
    f = pad_array(f, 1.)
    source_point_padded = (source_point[0] + 1, source_point[1] + 1)
    point_stages = 2 * np.ones(shape, dtype=int)
    W = np.full(shape=shape, fill_value=100.)
    point_stages = pad_array(point_stages, 0)
    W = pad_array(W, 100.)
    point_stages[source_point_padded] = 0
    W[source_point_padded] = 0
    i_star, j_star = source_point_padded
    while np.any(point_stages != 0):
        update_neighbours(i_star, j_star, point_stages, W, dxy, f)
        Trial = point_stages == 1
        index = np.argmin(np.where(Trial, W, np.inf))
        i_star, j_star = np.unravel_index(index, W.shape)
        point_stages[i_star, j_star] = 0
    return unpad_array(W)

def pad_array(array, fill_value):
    padded_shape = (array.shape[0] + 2, array.shape[1] + 2)
    padded_array = np.ones(padded_shape, dtype=array.dtype) * fill_value
    padded_array[1:-1, 1:-1] = array
    return padded_array

def unpad_array(padded_array):
    return padded_array[1:-1, 1:-1]

def update_neighbours(i_star, j_star, point_stages, W, dxy, f):
    neighbours = sees_point(i_star, j_star)
    for i, j in neighbours:
        if point_stages[i, j] != 0:
            point_stages[i, j] = 1
            update_W(i, j, W, dxy, f)

def update_W(i, j, W, dxy, f):
    Wx, Wy = gradient_W(i, j, W)
    if np.abs(Wx - Wy) >= dxy:
        W[i, j] = np.min((Wx + dxy / f[i, j], Wy + dxy / f[i, j]))
    else:
        W[i, j] = (Wx + Wy + np.sqrt((Wx + Wy) ** 2 - 2 * (Wx ** 2 + Wy ** 2 - (dxy / f[i, j]) ** 2))) / 2

def gradient_W(i, j, W):
    Wx = min(W[i + 1, j], W[i - 1, j])
    Wy = min(W[i, j + 1], W[i, j - 1])
    return Wx, Wy

def sees_point(i_star, j_star):
    return ((i_star + 1, j_star), (i_star - 1, j_star), (i_star, j_star + 1), (i_star, j_star - 1))

In [ ]:
if do_fast_marching:
    W_fast_marching = fast_marching_R2(cost, source_point, dxy)

In [ ]:
W_R2_flat[source_point[0]-1:source_point[0]+2, source_point[1]-1:source_point[1]+2]

In [ ]:
W_exact[source_point[0]-1:source_point[0]+2, source_point[1]-1:source_point[1]+2]

In [ ]:
if do_fast_marching:
    print(W_fast_marching[source_point[0]-1:source_point[0]+2, source_point[1]-1:source_point[1]+2])

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 5))
ax.contour(xs, ys, W_exact)
ax.scatter(*source_point_real, label="Source")
contour = ax.contour(xs, ys, W_R2_flat, linestyles="dashed")
if do_fast_marching:
    ax.contour(xs, ys, W_fast_marching, linestyles="dotted")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_aspect("equal")
ax.legend()
fig.colorbar(contour, label="$W(x, y)$");

#### Geodesic tracking
Notably, the coordinates of the geodesic do not depend on the metric tensor. To see this, note that we can find the geodesic via backtracking:
$$\begin{dcases} \dot{\gamma}(t) = -\nabla W(\gamma(t)), & t > 0, \\
\gamma(0) = (x^*, y^*). & \end{dcases}$$
Now, the gradient operator $\nabla$ depends on the metric tensor in such a way that the contributions cancel out. In particular, if the metric tensor is given (with respect to some coordinates) by
$$G = \begin{pmatrix} g_{xx} & g_{xy} \\
g_{xy} & g_{yy}\end{pmatrix},$$
then we can find that the inverse metric tensor is given (with respect to the dual coordinates) by
$$G^{-1} = \begin{pmatrix} g^{xx} & g^{xy} \\
g^{xy} & g^{yy}\end{pmatrix} = \frac{1}{g_{xx} g_{yy} - g_{xy}^2} \begin{pmatrix} g_{yy} & -g_{xy} \\
-g_{xy} & g_{xx}\end{pmatrix}.$$
It can be shown that the gradient, which is the Riesz representative of the differential, may be written as
$$\nabla W(x, y) = (g^{xx} \partial_x W(x, y) + g^{xy} \partial_y W(x, y), g^{xy} \partial_x W(\gamma(t)) + g^{yy} \partial_y W(x, y)).$$
Filling everything in, we see that 
$$\nabla W(x, y) = \frac{1}{W(x, y)} (x, y) \propto (x, y).$$
Hence, the orientation of the gradient does not depend on the metric tensor. We will therefore always find the same geodesic by steepest descent, up to some parametrisation.

Therefore, geodesics will just be straight lines. Since our source point is at the origin, this means that the gradient (expressed with respect to the coordinate frame) should be proportional to the target point (expressed with respect to its coordinate frame).

In [ ]:
target_point_real = (1.2, 0.5)
target_point = eikivp.R2.metric.coordinate_real_to_array(*target_point_real, x_min, y_max, dxy)
print(grad_W_R2_flat[target_point]) # given with respect to real coordinates? If so, // target_point_real
print(np.arctan2(grad_W_R2_flat[target_point][1], grad_W_R2_flat[target_point][0]), np.arctan2(target_point_real[1], target_point_real[0]))
print(np.arctan2(grad_W_R2_flat[target_point][1], grad_W_R2_flat[target_point][0]) - np.arctan2(target_point_real[1], target_point_real[0]))

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
ax.streamplot(np.flip(xs, axis=0), np.flip(ys, axis=0), -np.flip(grad_W_R2_flat[:, :, 0], axis=0), -np.flip(grad_W_R2_flat[:, :, 1], axis=0), color="red")
ax.scatter(*source_point_real, label="Source", marker=".")
ax.scatter(*target_point_real, label="Target", marker=".")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_aspect("equal")
ax.legend();

In [ ]:
orientations = np.arctan2(grad_W_R2_flat[:, :, 1], grad_W_R2_flat[:, :, 0])

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 5))
contour = ax.contourf(xs, ys, orientations)
ax.scatter(*source_point_real, label="Source", marker=".")
ax.scatter(*target_point_real, label="Target", marker=".")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_aspect("equal")
ax.legend()
fig.colorbar(contour, label="$\\angle \\nabla W(x, y)$");

In [ ]:
def get_grad_norms(grad_W_np, G_np, cost_np):
    grad_W = ti.Vector.field(n=grad_W_np.shape[-1], shape=grad_W_np.shape[:-1], dtype=ti.f32)
    grad_W.from_numpy(grad_W_np)
    grad_norm = ti.field(dtype=ti.f32, shape=grad_W.shape)
    G = ti.Matrix(G_np, ti.f32)
    cost = ti.field(dtype=ti.f32, shape=grad_W.shape)
    cost.from_numpy(cost_np)
    compute_grad_norm(grad_W, grad_norm, G, cost)
    return grad_norm.to_numpy()

@ti.kernel
def compute_grad_norm(
    grad_W: ti.template(),
    grad_norm: ti.template(),
    G: ti.types.matrix(2, 2, ti.f32),
    cost: ti.template()
):
    for I in ti.grouped(grad_norm):
        grad_norm[I] = eikivp.R2.metric.norm(grad_W[I], G, cost[I])

In [ ]:
grad_norms = get_grad_norms(grad_W_R2_flat, G_np, np.ones(W_R2_flat.shape))
grad_norms[source_point[0]-1:source_point[0]+2, source_point[1]-1:source_point[1]+2]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 5))
contour = ax.contourf(xs, ys, grad_norms + source_point_array - 1) # gradient at source is 0.
ax.scatter(*source_point_real, label="Source", marker=".")
ax.scatter(*target_point_real, label="Target", marker=".")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_aspect("equal")
ax.legend()
fig.colorbar(contour, label="$\\Vert \\nabla W(x, y) \\Vert$");

In [ ]:
spatial_stride = 20
begin_index = 0
end_index = -1
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 5))
scatter = ax.scatter(
    xs[begin_index::spatial_stride, begin_index::spatial_stride],
    ys[begin_index::spatial_stride, begin_index::spatial_stride],
    c=grad_norms[begin_index::spatial_stride, begin_index::spatial_stride] + source_point_array[begin_index::spatial_stride, begin_index::spatial_stride] - 1,
    cmap='PRGn'
)
ax.scatter(*source_point_real, label="Source", marker=".")
ax.scatter(*target_point_real, label="Target", marker=".")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_aspect("equal")
fig.colorbar(scatter, label="$\\Vert \\nabla W(x, y) \\Vert$");

In [ ]:
γ = eikivp.geodesic_back_tracking_R2(grad_W_R2_flat, source_point, target_point, cost, xs, ys, G_np=G_np, n_max=2000)

In [ ]:
(γ[1:, :] - γ[:-1, :])[:5] # difference should be roughly constant because geodesic should be straight line.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 5))
ax.contour(xs, ys, W_exact)
contour = ax.contour(xs, ys, W_R2_flat, linestyles="dashed")
if do_fast_marching:
    ax.contour(xs, ys, W_fast_marching, linestyles="dotted")
ax.scatter(*source_point_real, label="Source", marker=".")
ax.scatter(*target_point_real, label="Target", marker=".")
ax.arrow(*target_point_real, -grad_W_R2_flat[target_point][0], -grad_W_R2_flat[target_point][1])
ax.plot(γ[:, 0], γ[:, 1], label="Geodesic")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_aspect("equal")
ax.legend()
fig.colorbar(contour, label="$W(x, y)$");

### Retinal image
Computing the distance function on flat $\mathbb{R}^2$ is not very impressive. We would like to be able to do it also when the space is not flat, for instance in the case of a retinal image. To track the vessels in a retinal image, it makes sense to assign a low cost to those areas of the image that are vessels, and a high cost to those that aren't. We must therefore first compute such a cost function.

To do this, we use Frangi vesselness filtering, developed by Frangi et al. in ["Multiscale vessel enhancement filtering"](https://doi.org/10.1007/BFb0056195) (1998).

#### Frangi vesselness filtering
Let's first load in the image.

In [ ]:
ds = 8
retinal_image = Image.open("E46_OD_best.tif")
width, height = retinal_image.size
retinal_image_gray_ds = retinal_image.resize((width // ds, height // ds)).convert("L")
retinal_array_unnormalised = np.array(retinal_image_gray_ds).astype(np.float64)
retinal_array = eikivp.utils.image_rescale(retinal_array_unnormalised)
eikivp.visualisations.view_image_array(retinal_array);

We will already define the start and the end of our geodesic (`source_point` and `target_point`, respectively).

In [ ]:
dim_I, dim_J = retinal_array.shape
Is, Js = np.indices((dim_I, dim_J))
dxy = 1.
x_min = 0.
y_max = dim_I - 1
xs, ys = eikivp.R2.metric.coordinate_array_to_real(Is, Js, x_min, y_max, dxy)
x_max = xs[-1, -1]
y_min = ys[-1, -1]

In [ ]:
source_point = (246, 302)
target_point = (211, 118)
source_point_real = eikivp.R2.metric.coordinate_array_to_real(*source_point, x_min, y_max, dxy)
print(source_point_real)
source_point_array = (np.logical_and(xs == source_point_real[0], ys == source_point_real[1])) # 1 at source point, 0 else.
target_point_real = eikivp.R2.metric.coordinate_array_to_real(*target_point, x_min, y_max, dxy)
print(target_point_real)
G_np = np.identity(2)

We can now perform the Frangi filtering. The values for the scales, as well as the Frangi filter parameters $\alpha$, $\gamma$, and $\varepsilon$ were taken from the Mathematica notebook "Code A - Vesselness in SE(2).nb".

In [ ]:
scales = (np.array((2, 3, 4, 5), dtype=float)) / 4
vesselness_unmasked = eikivp.R2.vesselness.multiscale_frangi_filter(-retinal_array, scales, α=0.2, γ=3/4, ε=0.2)
mask = (retinal_array > 0) # Remove boundary
vesselness_unnormalised = vesselness_unmasked * sp.ndimage.binary_erosion(mask, iterations=int(np.ceil(scales.max() * 2)))
print(f"Before rescaling, vesselness is in [{vesselness_unnormalised.min()}, {vesselness_unnormalised.max()}].")
vesselness = eikivp.utils.image_rescale(vesselness_unnormalised)

In [ ]:
image_vesselness = eikivp.visualisations.convert_array_to_image(vesselness)
fig, ax = eikivp.visualisations.plot_image_array(image_vesselness, x_min, x_max, y_min, y_max)
ax.scatter(*source_point_real, label="Source", marker=".")
ax.scatter(*target_point_real, label="Target", marker=".")
ax.legend();

With these parameters, the vesselness score produced by the Frangi filter looks similar to the one in the Mathematica notebook "code A - Vesselness in SE(2).nb". However, the normalisation is not the same.

#### Cost function
Given the vesselness function, we compute the cost function as
$$\mathrm{cost}_{\lambda, p}(x, y) = \frac{1}{1 + \lambda \cdot \vert \mathrm{vesselness} \vert^p}$$
We choose $p = 2$, like in the Mathematica notebook, but $\lambda = 100$ instead of $\lambda = 1000$. This produces a cost function that looks more similar (probably due to the difference in normalisation).

In [ ]:
cost = eikivp.costfunction.cost_function(vesselness, 100, 2)

In [ ]:
image_cost = eikivp.visualisations.convert_array_to_image(cost)
fig, ax = eikivp.visualisations.plot_image_array(image_cost, x_min, x_max, y_min, y_max)
ax.scatter(*source_point_real, label="Source", marker=".")
ax.scatter(*target_point_real, label="Target", marker=".")
ax.legend();

#### Distance map
We are now able to compute the distance map, which is the viscosity solution of the Eikonal equation
$$\begin{cases} \Vert \nabla_{\mathrm{cost}} W(x, y) \Vert_{\mathrm{cost}} = 1, & (x, y) \in \mathbb{R}^2 \setminus \{(x_0, y_0)\}, \\
W(x, y) = 0, & (x, y) = (x_0, y_0), \end{cases}$$
where $\nabla_{\mathrm{cost}}$ is the datadriven gradient. In standard Euclidean $\mathbb{R}^2$, the datadriven gradient is given by
$$\nabla_{\mathrm{cost}} W(x, y) = \frac{1}{\mathrm{cost}_{\lambda, p}^2(x, y)} \nabla W(x, y) = \frac{1}{\mathrm{cost}_{\lambda, p}^2(x, y)} (\partial_x W(x, y), \partial_y W(x, y)),$$
and furthermore satisfies that
$$\Vert \nabla_{\mathrm{cost}} W(x, y) \Vert_{\mathrm{cost}} = \sqrt{\mathrm{cost}_{\lambda, p}^{-2}(x, y) (\vert \partial_x W(x, y) \vert^2 + \vert \partial_y W(x, y) \vert^2)}.$$
The quality of the approximation spreads out from the source point. Hence, if we are interested only in the distance map to points that are not that far away (in terms of the true distance map), then we do not have to perform as many iterations. __We terminate the iteration when the value of the approximate distance map changes less than a certain tolerance ($10^{-5}$) after a step. NOT ANYMORE__

In [ ]:
W_R2_retinal, grad_W_R2 = eikivp.eikonal_solver_R2(cost, source_point, dxy=dxy, n_max=40000)
# W_fast_marching = fast_marching_R2(cost) # Too slow...

If all is well, then the distance map should be $0$ at the source point, and vary nicely around the source_point.

In [ ]:
W_R2_retinal[source_point[0]-1:source_point[0]+2, source_point[1]-1:source_point[1]+2]

We can now visualise the distance map with a contour plot.

In [ ]:
fig, ax = eikivp.visualisations.plot_image_array(cost, x_min, x_max, y_min, y_max, figsize=(12, 10))
max_distance = np.round(W_R2_retinal[target_point] * 2.5)
contour = ax.contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax.scatter(*source_point_real, label="Source", marker=".")
ax.scatter(*target_point_real, label="Target", marker=".")
ax.legend()
fig.colorbar(contour, label="$W(x, y)$");

#### Geodesic tracking
Now that we have the distance map (from a certain source point), we are able to compute the geodesics between the source point and any other point $(x^*, y^*)$ using backtracking:
$$\begin{dcases} \dot{\gamma}(t) = -\nabla_{\mathrm{cost}} W(\gamma(t)), & t > 0, \\
\gamma(0) = (x^*, y^*). & \end{dcases}$$
We can numerically solve this using a Forward Euler discretisation:
$$\begin{dcases} \gamma_{n + 1} = \gamma_n - \Delta t \nabla_{\mathrm{cost}} W(\gamma(t)), & n > 0, \\
\gamma_0 = (x^*, y^*). & \end{dcases}$$

We have already computed the gradient field $\nabla_{\mathrm{cost}} W$, shown below.

In [ ]:
fig, ax = eikivp.visualisations.plot_image_array(vesselness, x_min, x_max, y_min, y_max)
ax.streamplot(np.flip(xs[100:-100, 100:-100], axis=0), np.flip(ys[100:-100, 100:-100], axis=0), -np.flip(grad_W_R2[100:-100, 100:-100, 0], axis=0), -np.flip(grad_W_R2[100:-100, 100:-100, 1], axis=0), color="red")
ax.scatter(*source_point_real, label="Source", marker=".")
ax.scatter(*target_point_real, label="Target", marker=".")
ax.legend();

In [ ]:
grad_norms = get_grad_norms(grad_W_R2, np.identity(2), cost)
grad_norms[source_point[0]-1:source_point[0]+2, source_point[1]-1:source_point[1]+2]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 10))
contour = ax.contourf(xs, ys, grad_norms + source_point_array - 1) # gradient at source is 0.
ax.scatter(*source_point_real, label="Source", marker=".")
ax.scatter(*target_point_real, label="Target", marker=".")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.legend()
fig.colorbar(contour, label="$\\Vert \\nabla W(x, y) \\Vert$");

In [ ]:
spatial_stride = 20
begin_index = 0
end_index = -1
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 10))
scatter = ax.scatter(
    xs[begin_index::spatial_stride, begin_index::spatial_stride],
    ys[begin_index::spatial_stride, begin_index::spatial_stride],
    c=grad_norms[begin_index::spatial_stride, begin_index::spatial_stride] + source_point_array[begin_index::spatial_stride, begin_index::spatial_stride] - 1,
    cmap='PRGn'
)
ax.scatter(*source_point_real, label="Source", marker=".")
ax.scatter(*target_point_real, label="Target", marker=".")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
fig.colorbar(scatter, label="$\\Vert \\nabla W(x, y) \\Vert$");

In [ ]:
γ = eikivp.geodesic_back_tracking_R2(grad_W_R2, source_point, target_point, cost, xs, ys, n_max=2000)

We can see from the visualisation below that the geodesic follows the vessel rather well. However, this vessel was quite easy to follow: there were no crossings where the backtracking in $\mathbb{R}^2$ could take shortcuts.

In [ ]:
fig, ax = eikivp.visualisations.plot_image_array(cost, x_min, x_max, y_min, y_max)
ax.plot(γ[:, 0], γ[:, 1], label="geodesic", color="green")
ax.scatter(*source_point_real, label="Source", marker=".")
ax.scatter(*target_point_real, label="Target", marker=".")
ax.arrow(*target_point_real, -grad_W_R2[target_point][0]/2, -grad_W_R2[target_point][1]/2, width=0.5, head_width=5, color="red", label="$\\nabla W(\\mathrm{target})$")
ax.legend();

In [ ]:
fig, ax = eikivp.visualisations.plot_image_array(cost, x_min, x_max, y_min, y_max)
ax.plot(γ[:, 0], γ[:, 1], label="geodesic", color="green")
ax.scatter(*source_point_real, label="Source", marker=".")
ax.scatter(*target_point_real, label="Target", marker=".")
ax.arrow(*target_point_real, -grad_W_R2[target_point][0]/2, -grad_W_R2[target_point][1]/2, width=0.5, head_width=5, color="red", label="$\\nabla W(\\mathrm{target})$")
ax.set_xlim(100, 320)
ax.set_ylim(120, 340)
ax.legend();

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(28, 10))
eikivp.visualisations.plot_image_array(cost, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
contour = ax[0].contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax[0].plot(γ[:, 0], γ[:, 1], label="geodesic", color="green")
ax[0].scatter(*source_point_real, label="Source", marker=".")
ax[0].scatter(*target_point_real, label="Target", marker=".")
ax[0].legend()
fig.colorbar(contour, label="$W(x, y)$")

eikivp.visualisations.plot_image_array(cost, x_min, x_max, y_min, y_max, fig=fig, ax=ax[1])
contour = ax[1].contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax[1].plot(γ[:, 0], γ[:, 1], label="geodesic", color="green")
ax[1].scatter(*source_point_real, label="Source", marker=".")
ax[1].scatter(*target_point_real, label="Target", marker=".")
ax[1].set_xlim(100, 320)
ax[1].set_ylim(120, 340)
ax[1].legend()
fig.colorbar(contour, label="$W(x, y)$");

In [ ]:
target_point_bad = (199, 179)
target_point_bad_real = eikivp.R2.metric.coordinate_array_to_real(*target_point_bad, x_min, y_max, dxy)
γ_bad = eikivp.geodesic_back_tracking_R2(grad_W_R2, source_point, target_point_bad, cost, xs, ys, n_max=2000)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(28, 10))
eikivp.visualisations.plot_image_array(cost, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
contour = ax[0].contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax[0].plot(γ_bad[:, 0], γ_bad[:, 1], label="geodesic", color="green")
ax[0].scatter(*source_point_real, label="Source", marker=".")
ax[0].scatter(*target_point_bad_real, label="Target", marker=".")
ax[0].legend()
fig.colorbar(contour, label="$W(x, y)$")

eikivp.visualisations.plot_image_array(cost, x_min, x_max, y_min, y_max, fig=fig, ax=ax[1])
contour = ax[1].contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax[1].plot(γ_bad[:, 0], γ_bad[:, 1], label="geodesic", color="green")
ax[1].scatter(*source_point_real, label="Source", marker=".")
ax[1].scatter(*target_point_bad_real, label="Target", marker=".")
ax[1].set_xlim(100, 320)
ax[1].set_ylim(120, 340)
ax[1].legend()
fig.colorbar(contour, label="$W(x, j)");

## $SE(2)$
We have now done vessel tracking in flat Euclidean space. Since $\mathbb{R}^2$ is isotropic, moving in any direction (before introducing a datadriven cost term) is equally expensive. Vessels, however, have a clear orientation - namely in the direction of the vessel - which typically does not drastically change as we move along the vessel. It therefore would make sense to penalise motions that are not in line with the vessel orientation. We can do this by modelling the retina with the special Euclidean group $SE(2) = \mathbb{R}^2 \rtimes S^1$, which can be interpreted as the space of positions and orientations on $\mathbb{R}^2$. 

### $SE(2)$ background

The space of positions and orientations is a Lie group, which means it is a smooth manifold with a group product. The group product on $SE(2)$ is given by
$$(\vec{x}_1, \theta_1) \cdot (\vec{x}_2, \theta_2) = (\vec{x}_1 + R_{\theta_1} \vec{x}_2, \theta_1 + \theta_2),$$
where 
$$R_{\theta} = \begin{pmatrix} \cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta) \end{pmatrix}$$
is the matrix corresponding to a rotation with angle $\theta$.

In fact, $\mathbb{R}^2$ is also a Lie group, with $+$ as group operation. We can identify derivatives at a point in $\mathbb{R}^2$ with vectors in the tangent space at that point. Of particular interest are left invariant derivatives, which are exactly those that commute with left actions. The left action operator $\mathcal{L}: G \times L^2(G) \to L^2(G)$ on a Lie group $G$ is defined by
$$\mathcal{L}_g U(h) = U(L_g^{-1} h) = U(g^{-1} h)$$
for all $g, h \in G$. On $\mathbb{R}^2$, this would look like
$$\mathcal{L}_{\vec{y}} U(\vec{x}) = U(\vec{x} - \vec{y}).$$
Then a differential operator $\mathcal{A}$ commutes with left actions if 
$$\mathcal{L}_g \circ \mathcal{A} = \mathcal{A} \circ \mathcal{L}_g$$
for all $g \in G$, or, more concretely, if
$$(\mathcal{L}_g \circ \mathcal{A})(U)(h) = (\mathcal{A} \circ \mathcal{L}_g)(U)(h)$$
for all $g \in G$ and all sufficiently smooth $U \in L^2(G)$. It can be shown that on $\mathbb{R}^2$, the left invariant derivatives are (constant) linear combinations of $\partial_x$ and $\partial_y$. On $SE(2)$, however, things are a bit different: there the left invariant derivatives are linear combinations of the following vector fields:
$$\begin{align*} \mathcal{A}_1|_{(\vec{x}, \theta)} & = \cos(\theta) \partial_x|_{(\vec{x}, \theta)} + \sin(\theta) \partial_y|_{(\vec{x}, \theta)}, \\
\mathcal{A}_2|_{(\vec{x}, \theta)} & = -\sin(\theta) \partial_x|_{(\vec{x}, \theta)} + \cos(\theta) \partial_y|_{(\vec{x}, \theta)}, \textrm{ and} \\
\mathcal{A}_3|_{(\vec{x}, \theta)} & = \partial_{\theta}|_{(\vec{x}, \theta)}. \end{align*}$$
$\mathcal{A}_1$ and $\mathcal{A}_2$, which describe derivatives in the spatial directions, now depend on the orientation $\theta$.

To finally define the gradient on $SE(2)$, we need one more component: the metric tensor (field) $\mathcal{G}$. At each point, the metric tensor defines an inner product on the tangent space. We are again interested in the left invariant versions. It then turns out that then the metric tensor may be written as
$$\mathcal{G}_{(\vec{x}, \theta)} = \sum_{i, j} g_{ij} \omega^i|_{(\vec{x}, \theta)} \otimes \omega^j|_{(\vec{x}, \theta)},$$
where $\omega^i$ is the dual of $\mathcal{A}_i$ (so $\omega^i(\mathcal{A}_j) = \delta_j^i$), and $g_{ij}$ is constant. The inverse metric tensor, $\mathcal{G}^{-1}$, then has components $g^{ij}$ satisfying 
$$\sum_j g^{ij} g_{kj} = \delta^i_k.$$
The gradient of a smooth function is then defined as the Riesz representative of the differential of that function:
$$\mathcal{G}_{(\vec{x}, \theta)}(\nabla f(\vec{x}, \theta), v|_{(\vec{x}, \theta)}) \overset{\textrm{Riesz}}{:=} \langle df(\vec{x}, \theta), v|_{(\vec{x}, \theta)}\rangle = \sum_i \mathcal{A}_i|_{(\vec{x}, \theta)} f d\omega^i|_{(\vec{x}, \theta)} \left(\sum_j v^j \mathcal{A}_j|_{(\vec{x}, \theta)}\right) = \sum_i v^i \mathcal{A}_i|_{(\vec{x}, \theta)} f.$$
Note that additionally we have 
$$\begin{align*} \mathcal{G}_{(\vec{x}, \theta)}(\nabla f(\vec{x}, \theta), v|_{(\vec{x}, \theta)}) & = \sum_{i, j} g_{ij} \omega^i|_{(\vec{x}, \theta)} \otimes \omega^j|_{(\vec{x}, \theta)}\left(\sum_k (\nabla f(\vec{x}, \theta))^k \mathcal{A}_k|_{(\vec{x}, \theta)}, \sum_l v^l \mathcal{A}_l|_{(\vec{x}, \theta)}\right) \\
& = \sum_{i, j} g_{ij} v^i (\nabla f(\vec{x}, \theta))^j\end{align*},$$
so that we may identify that
$$\mathcal{A}_i|_{(\vec{x}, \theta)} f = \sum_j g_{ij} (\nabla f(\vec{x}, \theta))^j.$$
Note now that 
$$\sum_j g_{ij} \left(\sum_k g^{kj} \mathcal{A}_k|_{(\vec{x}, \theta)} f\right) = \sum_k \delta^k_i \mathcal{A}_k|_{(\vec{x}, \theta)} f = \mathcal{A}_i|_{(\vec{x}, \theta)} f.$$
It therefore follows that 
$$(\nabla f(\vec{x}, \theta))^j = \sum_i g^{ij} \mathcal{A}_i|_{(\vec{x}, \theta)} f,$$
or, in other words,
$$\nabla f(\vec{x}, \theta) = \sum_i g^{ij} \mathcal{A}_i|_{(\vec{x}, \theta)} f \mathcal{A}_j|_{(\vec{x}, \theta)}.$$

The datadriven metric is then defined by pointwise multiplication with the cost function: 
$$\mathcal{G}_{(\vec{x}, \theta)}^{\mathrm{cost}} = \mathrm{cost}_{\lambda, p}^2(\vec{x}, \theta) \mathcal{G}_{(\vec{x}, \theta)}.$$
Clearly, the corresponding inverse metric is $\mathrm{cost}_{\lambda, p}^{-2}(\vec{x}, \theta) \mathcal{G}_{(\vec{x}, \theta)}^{-1}$. From this we can find the datadriven gradient
$$\nabla_{\mathrm{cost}} f(\vec{x}, \theta) = \sum_i g^{ij}_{\mathrm{cost}} \mathcal{A}_i|_{(\vec{x}, \theta)} f \mathcal{A}_j|_{(\vec{x}, \theta)} = \sum_i \frac{g^{ij}}{\mathrm{cost}_{\lambda, p}^2(\vec{x}, \theta)} \mathcal{A}_i|_{(\vec{x}, \theta)} f \mathcal{A}_j|_{(\vec{x}, \theta)},$$
satisfying that
$$\Vert \nabla_{\mathrm{cost}} f \Vert_{\mathrm{cost}}^2 = \sum_{ij} \frac{g^{ij}}{\mathrm{cost}_{\lambda, p}^2(\vec{x}, \theta)} \mathcal{A}_i|_{(\vec{x}, \theta)} f \mathcal{A}_j|_{(\vec{x}, \theta)} f.$$

### Flat space
To get a feel for it, we will again start by computing the distance map on flat space, when the cost function is identically equal to $1$. __(Now with plus controller) NOT YET__

#### Distance map
Here we define the coordinate system.

In [ ]:
x_min, x_max = -2*np.pi, 2*np.pi
y_min, y_max = -np.pi, np.pi
θ_min, θ_max = -np.pi, np.pi
N = 51
dim_J = 2*N-1
dim_I = N
dim_K = 33
if (x_max - x_min) / (dim_J - 1) == (y_max - y_min) / (dim_I - 1):
    dxy = (x_max - x_min) / (dim_J - 1)
else:
    raise ValueError("Spatial resolution in x- and y-directions is not the same.")
dθ = (θ_max - θ_min) / (dim_K - 1)
Is, Js, Ks = np.indices((dim_I, dim_J, dim_K))
xs, ys, θs = eikivp.SE2.metric.coordinate_array_to_real(Is, Js, Ks, x_min, y_max, θ_min, dxy, dθ)

In [ ]:
source_point_real = (0., 0., 0.)
source_point = eikivp.SE2.metric.coordinate_real_to_array(*source_point_real, x_min, y_max, θ_min, dxy, dθ)
source_point_array = np.zeros_like(xs)
source_point_array[source_point] = 1 # (np.logical_and(xs == source_point_real[0], ys == source_point_real[1], θs == source_point_real[2])) # 1 at source point, 0 else.
print(xs[source_point], ys[source_point], θs[source_point])
G_np = np.diag((1., 1e8, 2.))
cost = np.ones((dim_I, dim_J, dim_K))

In [ ]:
n = 5000
W_SE2_flat, grad_W_SE2_flat = eikivp.eikonal_solver_SE2(cost, source_point, G_np, dxy, θs, n_max=n)
print(W_SE2_flat.min(), W_SE2_flat.mean(), W_SE2_flat.max())
print(W_SE2_flat[source_point[0]-1:source_point[0]+2, source_point[1]-1:source_point[1]+2, source_point[2]-1:source_point[2]+2])
# W_SE2_flat_plus, grad_W_flat_plus = iterative_method_flat_SE2(N, N_ors, 4 * n, plus_controller=True)
# print(W_SE2_flat_plus.min(), W_SE2_flat_plus.mean(), W_SE2_flat_plus.max())
# print(W_SE2_flat_plus[99:102, 99:102, 15:18])

In [ ]:
fig, ax = plt.subplots()
ax.plot(xs[source_point[0], :, source_point[2]], W_SE2_flat[source_point[0], :, source_point[2]], label="Sub Riemannian")
ax.set_xlim(x_min, x_max)
ax.set_ylim(0., W_SE2_flat[source_point[0], 0, source_point[2]])
ax.set_xlabel("$x$")
ax.set_ylabel("$W(x, 0, 0)$")
ax.set_title("Comparison of Distance Maps Along $x$-Axis")
ax.legend();

To visualise the distance map, we will look at isosurfaces, i.e., surfaces in $SE(2)$ such that the distance is constant over the surface. Mesh approximations of these can be computed using the marching cubes algorithm, implemented e.g. in [scikit-image](https://github.com/scikit-image/scikit-image).

In [ ]:
from skimage.measure import marching_cubes, block_reduce
# Maybe use Plotly or Mayavi instead of matplotlib
%matplotlib widget

In [ ]:
W_surface_level = 4
verts_flat, faces_flat, normals_flat, values_flat = marching_cubes(W_SE2_flat, level=W_surface_level, step_size=3)
print(values_flat.min(), values_flat.mean(), values_flat.max(), values_flat.shape)
# verts_flat_plus, faces_flat_plus, normals_flat_plus, values_flat_plus = marching_cubes(W_SE2_flat_plus, level=4, step_size=8)
# print(values_flat_plus.min(), values_flat_plus.mean(), values_flat_plus.max(), values_flat_plus.shape)

In [ ]:
fig = plt.figure(figsize=(10, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
ax = fig.add_subplot(111, projection="3d")
ax.plot_trisurf(verts_flat[:, 1] * dxy + x_min, verts_flat[:, 0] * dxy + y_min, faces_flat, verts_flat[:, 2] * dθ + θ_min, label="Sub Riemannian", alpha=0.5)
# ax.plot_trisurf((2 * verts_flat_plus[:, 0] / N) - 1, (2 * verts_flat_plus[:, 1] / N) - 1, faces_flat_plus, (2 * np.pi * verts_flat_plus[:, 2] / N_ors) - np.pi, label="Plus Controller", alpha=0.5)
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_zlim(θ_min, θ_max)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("θ")
ax.set_aspect("equal")
ax.legend();

#### Geodesic tracking
__REWRITE FOR SE(2)__
Notably, the coordinates of the geodesic do not depend on the metric tensor. To see this, note that we can find the geodesic via backtracking:
$$\begin{dcases} \dot{\gamma}(t) = -\nabla W(\gamma(t)), & t > 0, \\
\gamma(0) = (x^*, y^*). & \end{dcases}$$
Now, the gradient operator $\nabla$ depends on the metric tensor in such a way that the contributions cancel out. In particular, if the metric tensor is given (with respect to some coordinates) by
$$G = \begin{pmatrix} g_{xx} & g_{xy} \\
g_{xy} & g_{yy}\end{pmatrix},$$
then we can find that the inverse metric tensor is given (with respect to the dual coordinates) by
$$G^{-1} = \begin{pmatrix} g^{xx} & g^{xy} \\
g^{xy} & g^{yy}\end{pmatrix} = \frac{1}{g_{xx} g_{yy} - g_{xy}^2} \begin{pmatrix} g_{yy} & -g_{xy} \\
-g_{xy} & g_{xx}\end{pmatrix}.$$
It can be shown that the gradient, which is the Riesz representative of the differential, may be written as
$$\nabla W(x, y) = (g^{xx} \partial_x W(x, y) + g^{xy} \partial_y W(x, y), g^{xy} \partial_x W(\gamma(t)) + g^{yy} \partial_y W(x, y)).$$
Filling everything in, we see that 
$$\nabla W(x, y) = \frac{1}{W(x, y)} (x, y) \propto (x, y).$$
Hence, the orientation of the gradient does not depend on the metric tensor. We will therefore always find the same geodesic by steepest descent, up to some parametrisation.

Therefore, geodesics will just be straight lines. Since our source point is at the origin, this means that the gradient (expressed with respect to the coordinate frame) should be proportional to the target point (expressed with respect to its coordinate frame).

In [ ]:
target_point_real = (2., 1., np.pi/4)
target_point = eikivp.SE2.metric.coordinate_real_to_array(*target_point_real, x_min, y_max, θ_min, dxy, dθ)
print(grad_W_SE2_flat[target_point])

In [ ]:
@ti.kernel
def get_grad_W_static(
    grad_W_LI: ti.template(),
    θs: ti.template(),
    grad_W_static:ti.template()
):
    eikivp.SE2.metric.vectorfield_LI_to_static(grad_W_LI, θs, grad_W_static)

In [ ]:
grad_W_SE2_ti = ti.Vector.field(n=3, dtype=ti.f32, shape=grad_W_SE2_flat.shape[:-1])
grad_W_SE2_ti.from_numpy(grad_W_SE2_flat)
grad_W_SE2_static_ti = ti.Vector.field(n=3, dtype=ti.f32, shape=grad_W_SE2_ti.shape)
θs_ti = ti.field(dtype=ti.f32, shape=θs.shape)
θs_ti.from_numpy(θs)
get_grad_W_static(grad_W_SE2_ti, θs_ti, grad_W_SE2_static_ti)
grad_W_SE2_static = grad_W_SE2_static_ti.to_numpy()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
ax.streamplot(np.flip(xs[:, :, source_point[2]], axis=0), np.flip(ys[:, :, source_point[2]], axis=0), -np.flip(grad_W_SE2_flat[:, :, source_point[2], 0], axis=0), -np.flip(grad_W_SE2_flat[:, :, source_point[2], 1], axis=0), color="red")
ax.scatter(*source_point_real[:-1], label="Source", marker=".")
ax.scatter(*target_point_real[:-1], label="Target", marker=".")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_aspect("equal")
ax.legend();

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
ax.streamplot(np.flip(xs[:, :, source_point[2]+5], axis=0), np.flip(ys[:, :, source_point[2]+5], axis=0), -np.flip(grad_W_SE2_static[:, :, source_point[2]+5, 0], axis=0), -np.flip(grad_W_SE2_static[:, :, source_point[2]+5, 1], axis=0), color="red")
ax.scatter(*source_point_real[:-1], label="Source", marker=".")
ax.scatter(*target_point_real[:-1], label="Target", marker=".")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_aspect("equal")
ax.legend();

In [ ]:
orientations = np.arctan2(grad_W_SE2_static[:, :, :, 1], grad_W_SE2_static[:, :, :, 0])

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 5))
contour = ax.contourf(xs[:, :, source_point[2]], ys[:, :, source_point[2]], orientations[:, :, source_point[2]])
ax.scatter(*source_point_real[:-1], label="Source", marker=".")
ax.scatter(*target_point_real[:-1], label="Target", marker=".")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_aspect("equal")
ax.legend()
fig.colorbar(contour, label="$\\angle \\nabla W(x, y)$");

In [ ]:
def get_grad_norms(grad_W_np, G_np, cost_np):
    grad_W = ti.Vector.field(n=grad_W_np.shape[-1], shape=grad_W_np.shape[:-1], dtype=ti.f32)
    grad_W.from_numpy(grad_W_np)
    grad_norm = ti.field(dtype=ti.f32, shape=grad_W.shape)
    G = ti.Matrix(G_np, ti.f32)
    cost = ti.field(dtype=ti.f32, shape=grad_W.shape)
    cost.from_numpy(cost_np)
    compute_grad_norm(grad_W, grad_norm, G, cost)
    return grad_norm.to_numpy()

@ti.kernel
def compute_grad_norm(
    grad_W: ti.template(),
    grad_norm: ti.template(),
    G: ti.types.matrix(3, 3, ti.f32),
    cost: ti.template()
):
    for I in ti.grouped(grad_norm):
        grad_norm[I] = eikivp.SE2.metric.norm_LI(grad_W[I], G, cost[I])

In [ ]:
grad_norms = get_grad_norms(grad_W_SE2_flat, G_np, cost)
grad_norms[source_point[0]-1:source_point[0]+2, source_point[1]-1:source_point[1]+2, source_point[2]-1:source_point[2]+2]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 5))
contour = ax.contourf(xs[:, :, source_point[2]], ys[:, :, source_point[2]], (grad_norms + source_point_array - 1)[:, :, source_point[2]]) # gradient at source is 0.
ax.scatter(*source_point_real[:-1], label="Source", marker=".")
ax.scatter(*target_point_real[:-1], label="Target", marker=".")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_aspect("equal")
ax.legend()
fig.colorbar(contour, label="$\\Vert \\nabla W(x, y) \\Vert$");

In [ ]:
spatial_stride = 25//5
orientational_stride = 4
begin_index = 0
end_index = -1
fig = plt.figure(figsize=(10, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
ax = fig.add_subplot(111, projection="3d")
scatter = ax.scatter(
    xs[begin_index::spatial_stride, begin_index::spatial_stride, ::orientational_stride],
    ys[begin_index::spatial_stride, begin_index::spatial_stride, ::orientational_stride],
    θs[begin_index::spatial_stride, begin_index::spatial_stride, ::orientational_stride],
    c=grad_norms[begin_index::spatial_stride, begin_index::spatial_stride, ::orientational_stride]+source_point_array[begin_index::spatial_stride, begin_index::spatial_stride, ::orientational_stride]-1,
    cmap='PRGn'
)
ax.scatter(*source_point_real, label="Source", marker=".")
# ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("θ")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_zlim(θ_min, θ_max)
ax.legend()
ax.set_aspect("equal")
fig.colorbar(scatter, ax=ax);

In [ ]:
γ = eikivp.geodesic_back_tracking_SE2(grad_W_SE2_flat, source_point, target_point, cost, xs, ys, θs, G_np, dt=1., n_max=20000)

In [ ]:
γ[:10]

In [ ]:
grad_W_SE2_static[target_point]

In [ ]:
grad_W_SE2_ti[target_point]

In [ ]:
np.cos(θs[target_point]), np.sin(θs[target_point])

In [ ]:
@ti.kernel
def compute_norm(
    vec: ti.types.vector(3, ti.f32),
    G: ti.types.matrix(3, 3, ti.f32),
    cost: ti.f32,
    θ: ti.f32
) -> ti.f32:
    return eikivp.SE2.metric.norm_static(vec, G, cost, θ)

def compute_the_norm(vec_field, G_np, cost_field_np, θs, point):
    vec = vec_field[point]
    G = ti.Matrix(G_np, ti.f32)
    cost_field = ti.field(dtype=ti.f32, shape=vec_field.shape)
    cost_field.from_numpy(cost_field_np)
    cost = cost_field[point]
    θ = θs[point]
    return compute_norm(vec, G, cost, θ)

In [ ]:
compute_the_norm(grad_W_SE2_static_ti, G_np, cost, θs_ti, target_point)

In [ ]:
@ti.kernel
def normalise_vec(
    vec: ti.types.vector(3, ti.f32),
    G: ti.types.matrix(3, 3, ti.f32),
    cost: ti.f32
) -> ti.types.vector(3, ti.f32):
    return eikivp.SE2.metric.normalise_LI(vec, G, cost)

def normalise_the_vec(vec_field, G_np, cost_field_np, point):
    vec = vec_field[point]
    G = ti.Matrix(G_np, ti.f32)
    cost_field = ti.field(dtype=ti.f32, shape=vec_field.shape)
    cost_field.from_numpy(cost_field_np)
    cost = cost_field[point]
    return normalise_vec(vec, G, cost)

In [ ]:
normalise_the_vec(grad_W_SE2_ti, G_np, cost, target_point)

In [ ]:
fig = plt.figure(figsize=(10, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
ax = fig.add_subplot(111, projection="3d")
ax.plot_trisurf(verts_flat[:, 1] * dxy + x_min, verts_flat[:, 0] * dxy + y_min, faces_flat, verts_flat[:, 2] * dθ + θ_min, label="Sub Riemannian", alpha=0.5)
# ax.plot_trisurf((2 * verts_flat_plus[:, 0] / N) - 1, (2 * verts_flat_plus[:, 1] / N) - 1, faces_flat_plus, (2 * np.pi * verts_flat_plus[:, 2] / N_ors) - np.pi, label="Plus Controller", alpha=0.5)
ax.plot(γ[:, 0], γ[:, 1], γ[:, 2], label="Geodesic")
ax.scatter(xs[source_point], ys[source_point], θs[source_point], label="Source")
ax.scatter(xs[target_point], ys[target_point], θs[target_point], label="Target")
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_zlim(θ_min, θ_max)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("θ")
ax.set_aspect("equal")
ax.legend();

Projected onto $\mathbb{R}^2$.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot(γ[:, 0], γ[:, 1], label="Geodesic")
ax.scatter(xs[source_point], ys[source_point], label="Source")
ax.scatter(xs[target_point], ys[target_point], label="target")
ax.arrow(xs[target_point], ys[target_point], -grad_W_SE2_static[target_point][0], -grad_W_SE2_static[target_point][1])
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.legend();

__GPU MEMORY STUFF__
GPU has 4 GB of memory (20 GB shared). I think TaiChi crashes if we want to use more. Each number is 4 bytes, imported cost function is 512 x 512 x 32 numbers so 0.5 GB. At any one time we have circa 15 objects of that size to store intermediate results, so that is about 7.5 GB.

If we downsample by a factor 2 in each direction, we only use about 1 GB.

In [ ]:
cost_big = np.load("costOR.npy") # + 0.01
cost = block_reduce(cost_big, block_size=(2, 2, 2), func=np.min)
source_point = np.unravel_index(cost.argmin(axis=None), cost.shape)

_, _, ax = eik.cleanarrays.view_image_array(cost.min(axis=-1))
ax.scatter(source_point[1], source_point[0], marker=".");

In [ ]:
fig, ax = plt.subplots()
ax.scatter(range(16), cost[source_point[0], source_point[1], :]);

In [ ]:
G_inv = np.array(((1 / 16, 0., 0.), (0., 0., 0.), (0., 0., 1.)))
W_SE2_py, grad_W_SE2_py = eik.solvers.eikonal_solver_SE2_LI(G_inv, cost, source_point, 1., n_max=50000)
W_SE2_py.min(), W_SE2_py.mean(), W_SE2_py.max()

In [ ]:
print(W_SE2_py[source_point[0]-1:source_point[0]+2, source_point[1]-1:source_point[1]+2, source_point[2]-1:source_point[2]+2])
W_SE2_min = W_SE2_py.min(axis=-1)
print(W_SE2_min[source_point[0]-1:source_point[0]+2, source_point[1]-1:source_point[1]+2])

In [ ]:
target_point = (68, 142, 7) # (45, 155, 5)
W_SE2_py[target_point[0], target_point[1], :]

In [ ]:
_, _, ax = eik.cleanarrays.view_image_array(W_SE2_min / W_SE2_min.max());
ax.scatter(source_point[1], source_point[0], marker=".")
ax.scatter(target_point[1], target_point[0], marker=".");

In [ ]:
i_min, i_max = 0, W_SE2_min.shape[0] - 1
j_min, j_max = 0, W_SE2_min.shape[1] - 1
xs, ys = np.meshgrid(np.arange(i_min, i_max + 1), np.arange(j_min, j_max + 1))

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
ax.imshow(cost.min(axis=-1), cmap="gray", origin="upper")
# max_distance = np.round(W_R2_retinal[target_point] * 2.5)
max_distance = 25
contour = ax.contour(xs, ys, W_SE2_min, levels=np.linspace(0., max_distance, 5))
ax.scatter(xs[source_point[:2]], ys[source_point[:2]], label="Source", marker=".")
# ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(j_min, j_max)
ax.set_ylim(i_max, i_min)
ax.legend()
fig.colorbar(contour, label="W[i, j]");

In [ ]:
xs_3d, ys_3d, θs_3d = np.meshgrid(np.arange(i_min, i_max + 1), np.arange(j_min, j_max + 1), np.pi * (np.arange(0, 16) - 8) / 8)

In [ ]:
fig = plt.figure(figsize=(10, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
ax = fig.add_subplot(111, projection="3d")
# ax.imshow(vesselness, cmap="gray", origin="upper")
spatial_stride = 8
orientational_stride = 2
begin_index = 80
end_index = -80
ax.quiver(
    xs_3d[begin_index:end_index:spatial_stride, begin_index:end_index:spatial_stride, ::orientational_stride], 
    ys_3d[begin_index:end_index:spatial_stride, begin_index:end_index:spatial_stride, ::orientational_stride], 
    θs_3d[begin_index:end_index:spatial_stride, begin_index:end_index:spatial_stride, ::orientational_stride],
    -grad_W_SE2_py[begin_index:end_index:spatial_stride, begin_index:end_index:spatial_stride, ::orientational_stride, 1], 
    -grad_W_SE2_py[begin_index:end_index:spatial_stride, begin_index:end_index:spatial_stride, ::orientational_stride, 0], 
    -grad_W_SE2_py[begin_index:end_index:spatial_stride, begin_index:end_index:spatial_stride, ::orientational_stride, 2]
)
# ax.streamplot(xs[100:-100, 100:-100], ys[100:-100, 100:-100], -grad_W_R2[100:-100, 100:-100, 1], -grad_W_R2[100:-100, 100:-100, 0], color="red")
# ax.scatter(xs[source_point], ys[source_point], label="Source", marker=".")
# ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(j_min, j_max)
ax.set_ylim(i_max, i_min);
# ax.legend();

In [ ]:
verts_py, faces_py, normals_py, values_py = marching_cubes(W_SE2_py, level=20, step_size=3)
values_py.min(), values_py.mean(), values_py.max(), values_py.shape

In [ ]:
fig = plt.figure(figsize=(10, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
ax = fig.add_subplot(111, projection="3d")
ax.plot_trisurf(verts_py[:, 1], verts_py[:, 0], faces_py, (2 * np.pi * verts_py[:, 2] / cost.shape[2]) - np.pi, label="Sub Riemannian", alpha=0.5)
ax.scatter(target_point[1], target_point[0], (2 * np.pi * target_point[2] / cost.shape[2]) - np.pi, marker="o", color="red", alpha=1)
# ax.scatter([54, 55, 56], [154, 154, 154], [8, 8, 8])
ax.set_zlim(-np.pi, np.pi)
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(j_min, j_max)
ax.set_ylim(i_max, i_min)
ax.set_zlabel("θ")
ax.legend();

In [ ]:
γ_ci = eik.geodesic_back_tracking_SE2(grad_W_SE2_py, source_point, target_point, dt=1., β=0.8, n_max=1000)
γ = eik.convert_continuous_indices_to_real_space_SE2(γ_ci, xs_3d, ys_3d, θs_3d)

In [ ]:
grad_W_SE2_py[target_point[0]-1:target_point[0]+2, target_point[1]-1:target_point[1]+2, target_point[2]-1:target_point[2]+2]

In [ ]:
γ_ci[-10:]

In [ ]:
grad_norm = np.sqrt(np.sum(grad_W_SE2_py ** 2, axis=-1)).min(axis=-1)

In [ ]:
grad_norm.shape

In [ ]:
source_point

In [ ]:
grad_norm[source_point[0]-1:source_point[0]+2, source_point[1]-1:source_point[1]+2]

In [ ]:
fig, ax = plt.subplots()
contour = ax.contourf(xs, ys, grad_norm, levels=(0, 0.5, 1., 1.5, 2.))
ax.scatter(source_point[1], source_point[0], label="Source", marker=".")
# ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(120, 160)
ax.set_ylim(100, 60)
# ax.legend()
fig.colorbar(contour);

We can see from the visualisation below that the geodesic follows the vessel rather well. However, this vessel was quite easy to follow: there were no crossings where the backtracking in $\mathbb{R}^2$ could take shortcuts.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False

ax.imshow(cost.min(axis=-1), cmap="gray")
# contour = ax.contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax.plot(γ[:, 0], γ[:, 1], label="geodesic", color="red")
ax.scatter(xs[source_point[:2]], ys[source_point[:2]], label="Source", marker=".")
ax.scatter(xs[target_point[:2]], ys[target_point[:2]], label="Target", marker=".")
ax.set_xlabel("j")
ax.set_ylabel("i")
# ax.set_xlim(100, 320)
# ax.set_ylim(340, 120)
ax.legend();
# fig.colorbar(contour, label="W[i, j]");

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False

ax[0].imshow(vesselness, cmap="gray")
contour = ax[0].contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax[0].plot(γ[:, 0], γ[:, 1], label="geodesic", color="red")
ax[0].scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax[0].scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax[0].set_xlabel("j")
ax[0].set_ylabel("i")
ax[0].set_xlim(j_min, j_max)
ax[0].set_ylim(i_max, i_min)
ax[0].legend()
fig.colorbar(contour, label="W[i, j]")

ax[1].imshow(vesselness, cmap="gray")
contour = ax[1].contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax[1].plot(γ[:, 0], γ[:, 1], label="geodesic", color="red")
ax[1].scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax[1].scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax[1].set_xlabel("j")
ax[1].set_ylabel("i")
ax[1].set_xlim(100, 320)
ax[1].set_ylim(340, 120)
ax[1].legend()
fig.colorbar(contour, label="W[i, j]");

In [ ]:
target_point_bad = (199, 179)
γ_ci_bad = eik.geodesic_back_tracking_R2(grad_W_R2, source_point, target_point_bad, dt=1., n_max=1000)
γ_bad = eik.convert_continuous_indices_to_real_space_R2(γ_ci_bad, xs, ys)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False

ax[0].imshow(vesselness, cmap="gray")
contour = ax[0].contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax[0].plot(γ_bad[:, 0], γ_bad[:, 1], label="geodesic", color="red")
ax[0].scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax[0].scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax[0].set_xlabel("j")
ax[0].set_ylabel("i")
ax[0].set_xlim(j_min, j_max)
ax[0].set_ylim(i_max, i_min)
ax[0].legend()
fig.colorbar(contour, label="W[i, j]")

ax[1].imshow(vesselness, cmap="gray")
contour = ax[1].contour(xs, ys, W_R2_retinal, levels=np.linspace(0., max_distance, 5))
ax[1].plot(γ_bad[:, 0], γ_bad[:, 1], label="geodesic", color="red")
ax[1].scatter(xs[source_point], ys[source_point], label="Source", marker=".")
ax[1].scatter(xs[target_point], ys[target_point], label="Target", marker=".")
ax[1].set_xlabel("j")
ax[1].set_ylabel("i")
ax[1].set_xlim(100, 320)
ax[1].set_ylim(340, 120)
ax[1].legend()
fig.colorbar(contour, label="W[i, j]");

### Retinal image
We would now like to process the image of the retina in $SE(2)$, since thereon crossing vessels will become untangled.

#### Vesselness
Let's first load in the image, and apply a high pass filter

In [ ]:
ds = 8
retinal_image = Image.open("E46_OD_best.tif")
width, height = retinal_image.size
retinal_image_gray_ds = retinal_image.resize((width // ds, height // ds)).convert("L")
retinal_array_gray_ds = np.array(retinal_image_gray_ds).astype(np.float64)
retinal_array_unnormalised = eik.cleanarrays.high_pass_filter(retinal_array_gray_ds, 2 * 32 / ds)
retinal_array = eik.cleanarrays.image_rescale(retinal_array_unnormalised)
eik.cleanarrays.view_image_array(retinal_array);

We will already define the start and the end of our geodesic (`source_point` and `target_point`, respectively).

In [ ]:
def convert_angle_to_index(θ, N_ors):
    return int(N_ors * (θ + np.pi) / (2 * np.pi)) // N_ors

In [ ]:
N_ors = 65
θ_source = -5 * np.pi / 8
θ_target = np.pi
source_point = (246, 302, convert_angle_to_index(θ_source, N_ors)) # "y", "x" so row, column.
target_point = (211, 118, convert_angle_to_index(θ_target, N_ors))
# target_point = (194, 250)
i_min, i_max = 0, retinal_array.shape[0] - 1
j_min, j_max = 0, retinal_array.shape[0] - 1
θ_min, θ_max = -np.pi, np.pi
xs, ys, θs = np.meshgrid(np.arange(i_min, i_max + 1), np.arange(j_min, j_max + 1), np.linspace(-np.pi, np.pi, N_ors))

We can now perform the Frangi filtering. The values for the scales, as well as the Frangi filter parameters $\alpha$, $\gamma$, and $\varepsilon$ were taken from the Mathematica notebook "Code A - Vesselness in SE(2).nb".

In [ ]:
scales = (np.array((2, 3, 4, 5), dtype=float)) / 4
vesselness_unmasked = eik.multiscale_frangi_filter_R2(-retinal_array, scales, α=0.2, γ=3/4, ε=0.2)
mask = (retinal_array_gray_ds > 0) # Remove boundary
vesselness_unnormalised = vesselness_unmasked * sp.ndimage.binary_erosion(mask, iterations=int(np.ceil(scales.max() * 2)))
print(f"Before rescaling, vesselness is in [{vesselness_unnormalised.min()}, {vesselness_unnormalised.max()}].")
vesselness = eik.cleanarrays.image_rescale(vesselness_unnormalised)

In [ ]:
image_vesselness = eik.cleanarrays.convert_array_to_image(vesselness)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
ax.imshow(image_vesselness, cmap="gray", origin="upper")
ax.scatter(xs[source_point], ys[source_point], label="Source", marker=".", color="blue")
ax.arrow(xs[source_point], ys[source_point], 20 * np.cos(θ_source), 20 * np.sin(θ_source), color="blue", head_width=5)
ax.scatter(xs[target_point], ys[target_point], label="Target", marker=".", color="red")
ax.arrow(xs[target_point], ys[target_point], 20 * np.cos(θ_target), 20 * np.sin(θ_target), color="red", head_width=5)
ax.set_xlabel("j")
ax.set_ylabel("i")
ax.set_xlim(j_min, j_max)
ax.set_ylim(i_max, i_min)
ax.legend();

With these parameters, the vesselness score produced by the Frangi filter looks similar to the one in the Mathematica notebook "code A - Vesselness in SE(2).nb". However, the normalisation is not the same.